# EDA y Sinónimos para Queries MongoDB

Este notebook demuestra cómo realizar un análisis exploratorio de datos (EDA) y cómo manejar sinónimos en la generación de queries MongoDB para cumplir los entregables de la semana 3.

In [1]:
# 1. Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

In [2]:
# 2. Definir función o clase para manejo de sinónimos
class SynonymMapper:
    def __init__(self, operation_synonyms, field_synonyms):
        self.operation_synonyms = operation_synonyms
        self.field_synonyms = field_synonyms
    
    def normalize_operation(self, op):
        op = op.lower()
        for canonical, synonyms in self.operation_synonyms.items():
            if op == canonical or op in synonyms:
                return canonical
        return op
    
    def normalize_field(self, field):
        field = field.lower().replace(' ', '')
        for canonical, synonyms in self.field_synonyms.items():
            if field == canonical.lower() or field in [s.replace(' ', '') for s in synonyms]:
                return canonical
        return field

In [3]:
# 3. Ejemplo de mapeo de sinónimos de operaciones
operation_synonyms = {
    'sum': ['suma', 'sumar', 'adicionar'],
    'group': ['agrupar', 'group by', 'agrupar por'],
    'project': ['proyectar', 'seleccionar', 'mostrar'],
    'sort': ['ordenar', 'ordenar por', 'sort'],
    'join': ['une', 'unir', 'join', 'fusionar', 'combinar']
}

# 4. Ejemplo de mapeo de sinónimos de campos
field_synonyms = {
    'nombre': ['name', 'nombres', 'nombre'],
    'apellido': ['surname', 'apellidos', 'apellido'],
    'departamento_id': ['departamentoid', 'id_departamento', 'departamento_id'],
    'departamento_nombre': ['departamento', 'nombre_departamento', 'departamento_nombre']
}

In [4]:
# 5. Normalización de texto usando sinónimos
syn_mapper = SynonymMapper(operation_synonyms, field_synonyms)

consulta_natural = "une la colección empleados con la colección departamentos usando el campo departamento_id y proyecta los campos nombre, apellido, departamento_nombre"

# Normalizar operaciones y campos
operacion = syn_mapper.normalize_operation('une')
campo1 = syn_mapper.normalize_field('nombre')
campo2 = syn_mapper.normalize_field('apellido')
campo3 = syn_mapper.normalize_field('departamento_nombre')

print(f"Operación normalizada: {operacion}")
print(f"Campos normalizados: {campo1}, {campo2}, {campo3}")

Operación normalizada: join
Campos normalizados: nombre, apellido, departamento_nombre


In [5]:
# 6. Validación de sinónimos en consulta de ejemplo
# Simulación de traducción de consulta natural a pipeline canónico
consulta = {
    'operacion': syn_mapper.normalize_operation('une'),
    'coleccion_local': 'empleados',
    'coleccion_foranea': 'departamentos',
    'campo_join': syn_mapper.normalize_field('departamento_id'),
    'proyeccion': [
        syn_mapper.normalize_field('nombre'),
        syn_mapper.normalize_field('apellido'),
        syn_mapper.normalize_field('departamento_nombre')
    ]
}

print("Consulta traducida a forma canónica:")
print(consulta)

Consulta traducida a forma canónica:
{'operacion': 'join', 'coleccion_local': 'empleados', 'coleccion_foranea': 'departamentos', 'campo_join': 'departamento_id', 'proyeccion': ['nombre', 'apellido', 'departamento_nombre']}


# Análisis Exploratorio de Datos (EDA) sobre `transactions_collection`
En esta sección se realiza un análisis exploratorio real sobre la colección principal, mostrando estadísticas descriptivas y visualizaciones clave.

In [6]:
# Cargar datos de ejemplo de transactions_collection
import json
import pandas as pd
with open('../datasets/transactions_collection.json', encoding='utf-8') as f:
    data = json.load(f)
df = pd.json_normalize(data['sample_documents'])
df.head()

,Date,Devices
0,2024-01-15T10:30:00Z,"[{'Id': 'DEV001', 'BranchCode': 'PE240', 'Serv..."


In [7]:
# Estadísticas descriptivas de montos de transacción
if 'Devices.ServicePoints.ShipOutCycles.Transactions.Total' in df.columns:
    print(df['Devices.ServicePoints.ShipOutCycles.Transactions.Total'].describe())
else:
    print('No se encontró la columna de montos de transacción.')

No se encontró la columna de montos de transacción.


In [8]:
# Visualización: Histograma de montos de transacción
import matplotlib.pyplot as plt
if 'Devices.ServicePoints.ShipOutCycles.Transactions.Total' in df.columns:
    plt.hist(df['Devices.ServicePoints.ShipOutCycles.Transactions.Total'].dropna(), bins=10, color='skyblue')
    plt.xlabel('Monto de Transacción')
    plt.ylabel('Frecuencia')
    plt.title('Distribución de Montos de Transacción')
    plt.show()
else:
    print('No se encontró la columna de montos de transacción.')

No se encontró la columna de montos de transacción.
